In [1]:
from pdf2image import convert_from_path
import os
import re
import unicodedata
from collections import defaultdict
import difflib
import pandas as pd
from PIL import Image
import pytesseract
import cv2
import numpy as np
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

d:\Data\HCMUS\Y4\Y4_S1\Nhap_mon_xu_ly_ngon_ngu_tu_nhien\Mid_term\venv\Lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


## Tiền xử lý ảnh cho OCR Tesseract

In [2]:
INPUT_ROOT = '../data/image'
INPUT_SUBDIR = 'PDF1'
INPUT_DIR = os.path.join(INPUT_ROOT, INPUT_SUBDIR)
OUTPUT_SUBDIR = f"processed_{INPUT_SUBDIR}"
OUTPUT_DIR = os.path.join(INPUT_ROOT, OUTPUT_SUBDIR)
START_PAGE = 203
END_PAGE = 250

os.makedirs(OUTPUT_DIR, exist_ok=True)


# Lọc file trong phạm vi trang 
target_files = []
try:
    all_image_files = os.listdir(INPUT_DIR)
    pattern = re.compile(r'PDF1-(\d+)\.png$', re.IGNORECASE)
    
    for file_name in all_image_files:
        match = pattern.match(file_name)
        if match:
            page_number = int(match.group(1))
            if START_PAGE <= page_number <= END_PAGE:
                target_files.append(file_name)
                
except FileNotFoundError:
    print(f"Không tìm thấy thư mục đầu vào tại {INPUT_DIR}. ")
    
target_files.sort(key=lambda f: int(re.search(r'PDF1-(\d+)\.png$', f).group(1)))

print(f" Tìm thấy {len(target_files)} file ảnh (từ {START_PAGE} đến {END_PAGE}) cần xử lý.")


# Tiền xử lý ảnh OCR
def preprocess_for_ocr_enforce_white_bg(image_path):
    """
    Tiền xử lý nâng cao: Làm sắc nét, tăng tương phản, nhị phân hóa (Otsu), 
    khử nhiễu bằng Morphological Closing và đầu ra là NỀN TRẮNG/CHỮ ĐEN.
    """
    
    img = cv2.imread(image_path, cv2.IMREAD_COLOR) 
    if img is None:
        print(f"Không thể đọc ảnh tại {image_path}")
        return None, None
        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Làm Sắc Nét (Sharpening)
    sharpen_kernel = np.array([[ 0, -1,  0],
                               [-1,  5, -1],
                               [ 0, -1,  0]])
    sharpened = cv2.filter2D(gray, -1, sharpen_kernel)

    # Tăng Tương Phản (Gamma Correction)
    gamma = 2.0 
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    contrasted = cv2.LUT(sharpened, table)
    
    # Phân Ngưỡng Nhị Phân (Tách Nền/Chữ)
    
    # TH1: NỀN TRẮNG / CHỮ ĐEN (THRESH_BINARY_INV)
    _, thresh_inverted = cv2.threshold(contrasted, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # TH2: NỀN ĐEN / CHỮ TRẮNG (THRESH_BINARY)
    _, thresh_normal = cv2.threshold(contrasted, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Lựa chọn ảnh có nền trắng (nền chiếm ưu thế trong văn bản)
    # Ảnh có nền trắng sẽ có tổng số pixel trắng (255) lớn hơn đen (0) -> giá trị trung bình > 127
    if np.mean(thresh_inverted) > np.mean(thresh_normal):
        # thresh_inverted đã cho ra Nền Trắng / Chữ Đen (Định dạng mong muốn)
        binary_img = thresh_inverted
    else:
        # thresh_normal đã cho ra Nền Trắng / Chữ Đen (Ảnh gốc bị ngược màu)
        # Hoặc thresh_inverted cho ra Nền Đen / Chữ Trắng (Ảnh gốc xuôi màu)
        # Đảo ngược thresh_normal để có Nền Trắng / Chữ Đen
        binary_img = cv2.bitwise_not(thresh_normal)

    # Ép buộc về NỀN TRẮNG/CHỮ ĐEN để khử nhiễu

    if np.mean(binary_img) > 127:
        # Nền Trắng/Chữ Đen -> Đảo thành Nền Đen/Chữ Trắng (Tạm thời)
        img_for_denoise = cv2.bitwise_not(binary_img)
    else:
        img_for_denoise = binary_img
        
    # Khử Nhiễu Đốm Đen (Morphological Closing)
    # Dùng phép Đóng (Closing) để lấp đầy các đốm đen (nhiễu) trên nền trắng
    kernel = np.ones((2, 2), np.uint8) 
    denoised_closed = cv2.morphologyEx(img_for_denoise, cv2.MORPH_CLOSE, kernel)
    
    # Đảo Ngược Lại (Reverse) để có Nền Trắng/Chữ Đen
    final_img = cv2.bitwise_not(denoised_closed)

    # Kiểm tra lần cuối: Nếu vẫn là nền đen (do ảnh có quá nhiều chi tiết) -> đảo lại
    if np.mean(final_img) < 127: 
         final_img = cv2.bitwise_not(final_img)
         
    return final_img, img

# Xử lý và hiển thị kết quả cho ảnh mẫu
# sample_file = 'PDF1-203.png'

# if sample_file in target_files:
#     sample_path = os.path.join(INPUT_DIR, sample_file)
#     print(f" Tiền xử lý ảnh mẫu để hiển thị: {sample_file}")
#     processed_sample, original_img = preprocess_for_ocr_enforce_white_bg(sample_path)
    
#     if processed_sample is not None:
#         output_path = os.path.join(OUTPUT_DIR, f"processed_{sample_file}")
#         cv2.imwrite(output_path, processed_sample)
        
#         original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
        
#         plt.figure(figsize=(16, 8))
        
#         plt.subplot(1, 2, 1)
#         plt.imshow(original_img_rgb)
#         plt.title('1. Ảnh Gốc ')
#         plt.axis('off')
        
#         plt.subplot(1, 2, 2)
#         plt.imshow(processed_sample, cmap='gray') 
#         plt.title('2. Ảnh Đã Xử Lý (Nền Trắng/Chữ Đen)')
#         plt.axis('off')
        
#         plt.tight_layout()
#         plt.show()
    
#     print("-" * 70)
# else:
#     print(f" Cảnh báo: Không tìm thấy file mẫu '{sample_file}' trong thư mục đã lọc.")

print(f"Xử lý và lưu {len(target_files)} file ảnh")
count = 0
for file_name in target_files:
    full_path = os.path.join(INPUT_DIR, file_name)
    processed_img, _ = preprocess_for_ocr_enforce_white_bg(full_path)
    
    if processed_img is not None:
        output_file_name = f"processed_{file_name}"
        output_path = os.path.join(OUTPUT_DIR, output_file_name)
        cv2.imwrite(output_path, processed_img)
        count += 1

print(f" Xử lý {count} file ảnh xong. Các file đã lưu vào thư mục: {OUTPUT_DIR}")

 Tìm thấy 48 file ảnh (từ 203 đến 250) cần xử lý.
Xử lý và lưu 48 file ảnh
 Xử lý 48 file ảnh xong. Các file đã lưu vào thư mục: ../data/image\processed_PDF1


## OCR bằng Tesseract và Paddle

In [3]:
# OCR Tesseract
TESSERACT_LANGS = 'chi_sim+chi_tra+vie'
def tesseract_ocr(img_path: str, lang: str = TESSERACT_LANGS):
    try:
        text_raw = pytesseract.image_to_string(
            Image.open(img_path),
            lang=lang,
            config='--psm 3'
        )
        return [l.strip() for l in text_raw.split('\n') if l.strip()]
    except Exception as e:
        print(f"Lỗi OCR {img_path}: {e}")
        return []
    
# OCR Paddle
ocr_paddle = PaddleOCR(
    use_angle_cls=True,
    lang='ch',
    show_log=False,
    use_gpu=True,
    gpu_id=0,
    gpu_mem=4096
)

In [4]:
def strip_accents(s: str) -> str:
    return "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )

def is_chinese(text: str) -> bool:
    if not text:
        return False
        
    pattern = (
        r'^['
        # Nhóm Hán tự
        r'\u4E00-\u9FFF'  # CJK Unified Ideographs (Phổ thông)
        r'\u3400-\u4DBF'  # CJK Extension A
        r'\uF900-\uFAFF'  # CJK Compatibility
        
        # Nhóm dấu câu CJK
        r'\u3000-\u303F'  # Dấu câu TQ (。 、 【 】...)
        r'\uFF00-\uFFEF'  # Dấu câu Full-width (！, ？, ０-９...)
        r'\uFE30-\uFE4F'  # CJK Compatibility Forms
        
        # Nhóm ASCII (không có kí tự Latin)
        r'\u0020-\u0040'  # Khoảng trắng, Dấu câu (!..@), SỐ (0-9)
        r'\u005B-\u0060'  # Dấu câu ( [ .. ` )
        r'\u007B-\u007E'  # Dấu câu ( { .. ~ )
        
        # Nhóm Latin-1 Suplement (Giữ lại để bắt dấu » « ©)
        # r'\u00A0-\u00FF'  
        r']+$'
    )
    
    return bool(re.fullmatch(pattern, text))

def is_page_number_line(text: str) -> bool:
    return re.fullmatch(r"^\s*\d{1,4}\s*$", text) is not None

def looks_like_vietnamese(text: str) -> bool:
    norm = strip_accents(text).lower()
    tokens = re.findall(r"[a-z0-9]+", norm)

    def has_similar(target: str, thr: float) -> bool:
        for t in tokens:
            if difflib.SequenceMatcher(None, t, target).ratio() >= thr:
                return True
        return False

    return (
        has_similar("buc", 0.5) and
        has_similar("thu", 0.5) and
        has_similar("viet", 0.5) and
        has_similar("cho", 0.5) and
        has_similar("chinh", 0.5) and
        has_similar("minh", 0.5)
    )

def extract_id(text: str):
    def norm_digits(tok: str):
        tok = tok.strip()
        tok = tok.replace('O', '0').replace('o', '0')
        tok = tok.replace('I', '1').replace('l', '1').replace('|', '1').replace('!', '1')

        digits = "".join(re.findall(r"\d+", tok))  # "50 4" -> "504"
        if not digits:
            return None
        if len(digits) < 3:
            return None
        return digits

    # Chinese: 第xxx 
    m = re.search(r"第\s*([0-9OoIl|!\s]{1,20})", text)
    if m:
        return norm_digits(m.group(1))

    # Vietnamese: so/s0/s6 ...
    norm = strip_accents(text).lower()
    m = re.search(r"(?:\bso\b|\bs0\b|\bs6\b|s[0o6])\s*([0-9OoIl|!\s]{1,20})", norm)
    if m:
        return norm_digits(m.group(1))

    return None

In [5]:
# Parse ảnh -> Data
DATA_MAP_TESSERACT = defaultdict(lambda: {"src": [], "tgt": [], "vi_started": False})
DATA_MAP_PADDLE = defaultdict(lambda: {"src": [], "tgt": [], "vi_started": False})
STATE_TESSERACT = {"current_id": None, "mode": None, "pending_src": []}
STATE_PADDLE = {"current_id": None, "mode": None, "pending_src": []}

def process_image_to_data(img_path: str, data_map, state, ocr_model):
    if ocr_model == 'Tesseract':
        lines = tesseract_ocr(img_path)
    else:
        result = ocr_paddle.ocr(img_path, cls=True)
        if not result or result[0] is None:
            return

        blocks = result[0]
        blocks = sorted(
            blocks,
            key=lambda b: (min(p[1] for p in b[0]), min(p[0] for p in b[0]))
        )
        lines = [b[1][0].strip() for b in blocks if b and b[1] and b[1][0]]

    current_id = state["current_id"]
    mode = state["mode"]
    pending_src = state["pending_src"]

    for text in lines:
        if not text:
            continue
        if is_page_number_line(text):
            continue
        
        # print(f"Model: {ocr_model}\n{text}")

        # Title tiếng Việt -> mode=vi (lấy ID) và chỉ xử lý như title nếu extract được ID
        if looks_like_vietnamese(text):
            maybe_id = extract_id(text)
            if maybe_id:
                # Title thật
                current_id = maybe_id
                _ = data_map[current_id]
                if pending_src:
                    data_map[current_id]["src"].extend(pending_src)
                    pending_src.clear()

                mode = "vi"
                if not data_map[current_id]["vi_started"]:
                    data_map[current_id]["tgt"] = []
                    data_map[current_id]["vi_started"] = True
                
                data_map[current_id]["tgt"].append(
                    f"Bức thư viết cho chính mình số {current_id}"
                )
                continue

        # Anchor ID (ưu tiên từ tiếng Trung: 第...)
        found_id = extract_id(text)
        if found_id:
            current_id = found_id
            _ = data_map[current_id]
            if pending_src:
                data_map[current_id]["src"].extend(pending_src)
                pending_src.clear()

            mode = "zh" if is_chinese(text) else "vi"
            if mode == "vi" and (not data_map[current_id]["vi_started"]):
                data_map[current_id]["tgt"] = []
                data_map[current_id]["vi_started"] = True
            
            data_map[current_id]["src"].append(
                f"写 给 自己 的 第 {current_id} 封 信"
            )
            continue

        # Chưa có ID -> giữ tiếng Trung để chờ, bỏ qua Pinyin
        if current_id is None:
            if is_chinese(text):
                pending_src.append(text)
            continue

        # Gom nội dung theo mode
        if is_chinese(text):
            if mode == "vi":
                # Gặp tiếng Trung sau tiếng Việt -> Reset để chờ ID mới
                pending_src.append(text)
                current_id = None
                mode = "zh"
            else:
                data_map[current_id]["src"].append(text)
        else:
            if mode == "vi":
                data_map[current_id]["tgt"].append(text)

    state["current_id"] = current_id
    state["mode"] = mode
    state["pending_src"] = pending_src

In [6]:
# Main process
IMG_DIR = "../data/image"
OUT_DIR = "../output/ocr"
os.makedirs(OUT_DIR, exist_ok=True)

for p in range(START_PAGE, END_PAGE + 1):
    img_path_tesseract = os.path.join(IMG_DIR, f"processed_{INPUT_SUBDIR}/processed_PDF1-{p}.png")
    img_path_paddle = os.path.join(IMG_DIR, f"{INPUT_SUBDIR}/PDF1-{p}.png")
    if os.path.exists(img_path_paddle):
        print(f"Đang xử lý: {img_path_paddle}...")
        process_image_to_data(img_path_tesseract, DATA_MAP_TESSERACT, STATE_TESSERACT, 'Tesseract')
        process_image_to_data(img_path_paddle, DATA_MAP_PADDLE, STATE_PADDLE, 'Paddle')

rows = []
all_ids = set(DATA_MAP_PADDLE.keys()) | set(DATA_MAP_TESSERACT.keys())

def sort_key(x):
    try:
        return int(str(x))
    except:
        return str(x)

for sid in sorted(all_ids, key=sort_key):
    # Lấy Tiếng Trung (src) từ PADDLE Map
    paddle_data = DATA_MAP_PADDLE.get(sid, {"src": [], "tgt": []})
    src_text = " ".join(paddle_data["src"]).strip()
    
    # Lấy Tiếng Việt (tgt) từ TESSERACT Map
    tesseract_data = DATA_MAP_TESSERACT.get(sid, {"src": [], "tgt": []})
    tgt_text = " ".join(tesseract_data["tgt"]).strip()
    # print(f"{sid}\n")
    # print(f"{src_text}\n")
    # print(f"{tgt_text}\n")
    if src_text or tgt_text:
        rows.append({
            "src_id": sid,
            "src_lang": src_text,  # src: Paddle
            "tgt_lang": tgt_text   # tgt: Tesseract
        })
df = pd.DataFrame(rows)

# Lọc ID 
ID_START = 501
ID_END = 625
df = df[df['src_id'].str.isdigit()].copy()
df['src_id_int'] = df['src_id'].astype(int)
df = df[(df['src_id_int'] >= ID_START) & (df['src_id_int'] <= ID_END)]
df = df.sort_values('src_id_int').drop(columns=['src_id_int']).reset_index(drop=True)


Đang xử lý: ../data/image\PDF1/PDF1-203.png...
Đang xử lý: ../data/image\PDF1/PDF1-204.png...
Đang xử lý: ../data/image\PDF1/PDF1-205.png...
Đang xử lý: ../data/image\PDF1/PDF1-206.png...
Đang xử lý: ../data/image\PDF1/PDF1-207.png...
Đang xử lý: ../data/image\PDF1/PDF1-208.png...
Đang xử lý: ../data/image\PDF1/PDF1-209.png...
Đang xử lý: ../data/image\PDF1/PDF1-210.png...
Đang xử lý: ../data/image\PDF1/PDF1-211.png...
Đang xử lý: ../data/image\PDF1/PDF1-212.png...
Đang xử lý: ../data/image\PDF1/PDF1-213.png...
Đang xử lý: ../data/image\PDF1/PDF1-214.png...
Đang xử lý: ../data/image\PDF1/PDF1-215.png...
Đang xử lý: ../data/image\PDF1/PDF1-216.png...
Đang xử lý: ../data/image\PDF1/PDF1-217.png...
Đang xử lý: ../data/image\PDF1/PDF1-218.png...
Đang xử lý: ../data/image\PDF1/PDF1-219.png...
Đang xử lý: ../data/image\PDF1/PDF1-220.png...
Đang xử lý: ../data/image\PDF1/PDF1-221.png...
Đang xử lý: ../data/image\PDF1/PDF1-222.png...
Đang xử lý: ../data/image\PDF1/PDF1-223.png...
Đang xử lý: .

## Hậu xử lý (post-processing)

In [7]:
# Định nghĩa bộ lọc Regex
def is_vietnamese(text):
    if not isinstance(text, str) or text.strip() == "":
        return False
    
    # \u0020-\u007E: Bao gồm a-z, A-Z, 0-9, dấu câu chuẩn (!, @, #, ., ?, ...)
    # Phần còn lại: Các nguyên âm có dấu và chữ Đ/đ của tiếng Việt
    vietnamese_chars = (
        "àáạảãâầấậẩẫăằắặẳẵ"
        "èéẹẻẽêềếệểễ"
        "ìíịỉĩ"
        "òóọỏõôồốộổỗơờớợởỡ"
        "ùúụủũưừứựửữ"
        "ỳýỵỷỹ"
        "đ"
        "ÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴ"
        "ÈÉẸẺẼÊỀẾỆỂỄ"
        "ÌÍỊỈĨ"
        "ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ"
        "ÙÚỤỦŨƯỪỨỰỬỮ"
        "ỲÝỴỶỸ"
        "Đ"
    )
    
    # Logic: Chuỗi hợp lệ chỉ được chứa các ký tự nằm trong 2 nhóm này
    pattern = f'^[\u0020-\u007E{vietnamese_chars}]+$'
    
    return bool(re.match(pattern, text))

# Drop các dòng bị thiếu (NaN) hoặc rỗng ở 1 trong 2 cột
# Chuyển chuỗi chỉ có khoảng trắng thành NaN 
df = df.replace(r'^\s*$', float('nan'), regex=True)
df.dropna(subset=['src_lang', 'tgt_lang'], inplace=True)

# Logic: Giữ lại dòng khi (Check Trung == True) và (Check Việt == True)
# Các dòng False sẽ bị loại bỏ
df_clean = df[
    df['src_lang'].apply(is_chinese) & 
    df['tgt_lang'].apply(is_vietnamese)
].copy()
df_clean.reset_index(drop=True, inplace=True)


In [8]:
# Xuất CSV
if not df.empty:
    min_id = df_clean['src_id'].astype(int).min()
    max_id = df_clean['src_id'].astype(int).max()
else:
    min_id = max_id = 0

out_csv = os.path.join(OUT_DIR, f"PDF1_{min_id}_{max_id}.csv")
df_clean.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("Saved:", out_csv)

Saved: ../output/ocr\PDF1_501_625.csv
